In [1]:
from tensorflow import keras
import numpy as np
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from HW8functions import Subject, HCPData, Cleaner, ModeKeys, Evaluator, CLASS_NAMES
import SimpleITK as sitk
INPUT_SHAPE = [None, 128, 128, 128, 1]


In [2]:
dataset = HCPData()

In [3]:
# dataset.batch_size = 100
# dataset.init_batches(ModeKeys.TRAIN)
# train_images, train_labels = dataset.next_batch()

In [4]:
dataset.batch_size = 10
dataset.init_batches(ModeKeys.TEST)
val_images, val_labels = dataset.next_batch()

Loading & cleaning batch, this may be a moment...
Loading Subject #0100
Loading Subject #0101
Loading Subject #0102
Loading Subject #0103
Loading Subject #0104
Loading Subject #0105
Loading Subject #0106
Loading Subject #0107
Loading Subject #0108
Loading Subject #0109


In [5]:
# train_images.shape, train_labels.shape

In [2]:
model = keras.Sequential()

# input_layer
model.add(keras.layers.Conv3D(filters=32, kernel_size=3, padding='same', data_format='channels_last', input_shape=INPUT_SHAPE[1:]))
# a bunch of conv layers
for i in range(3):
    model.add(keras.layers.Conv3D(filters=32, kernel_size=3, padding='same', data_format='channels_last', activation='relu'))
    model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Conv3D(filters=1, kernel_size=3, padding='same', data_format='channels_last'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 128, 128, 128, 32) 896       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 128, 128, 128, 32) 27680     
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 128, 32) 0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 128, 128, 128, 32) 27680     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 128, 32) 0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 128, 128, 128, 32) 27680     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128, 128, 128, 32) 0         
__________

In [5]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy', keras.metrics.MAE],
)

TypeError: mean_squared_error() missing 2 required positional arguments: 'y_true' and 'y_pred'

In [35]:
model_dir = os.path.join('/localscratch/Users/abpwrs', "models","STRAIGHT_CONV_20190528")
tboard_dir = os.path.join('/localscratch/Users/abpwrs', 'logs','STRAIGHT_CONV_20190528')
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
if not os.path.isdir(tboard_dir):
    os.mkdir(tboard_dir)

model_checkpoint = ModelCheckpoint(os.path.join(model_dir,"weights.{epoch:02d}-{val_loss:.2f}.hdf5"), save_best_only=True)
tensor_board = TensorBoard(log_dir=tboard_dir, write_graph=True, write_images=True)

## TRAINING DONE IN PYTHON FILE

In [36]:
# model.fit(
#     x=train_images,
#     y=train_labels,
#     batch_size=1,
#     verbose=1,
#     epochs=1000,
#     validation_data=(val_images, val_labels),
#     callbacks=[model_checkpoint, tensor_board],
#     use_multiprocessing=False
# )

Train on 1 samples, validate on 1 samples
Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 797.4623 - acc: 0.6040 - mean_absolute_error: 15.6558 - val_loss: 194.4658 - val_acc: 0.4677 - val_mean_absolute_error: 8.8340
Epoch 2/1000
1/1 [==============================] - 1s 790ms/step - loss: 712.8325 - acc: 0.5437 - mean_absolute_error: 14.7499 - val_loss: 48.0392 - val_acc: 0.7579 - val_mean_absolute_error: 4.4334
Epoch 3/1000
1/1 [==============================] - 1s 755ms/step - loss: 800.7390 - acc: 0.6175 - mean_absolute_error: 15.5309 - val_loss: 20.3266 - val_acc: 0.6218 - val_mean_absolute_error: 2.8058
Epoch 4/1000
1/1 [==============================] - 1s 746ms/step - loss: 495.6744 - acc: 0.5887 - mean_absolute_error: 12.3580 - val_loss: 64.9192 - val_acc: 0.7603 - val_mean_absolute_error: 5.1493
Epoch 5/1000
1/1 [==============================] - 1s 738ms/step - loss: 507.8427 - acc: 0.6389 - mean_absolute_error: 12.4603 - val_loss: 5.7908 - val_acc: 0.

KeyboardInterrupt: 

## Writing out of validation images

In [4]:
model.load_weights("/localscratch/Users/abpwrs/models/STRAIGHT_CONV_20190528/weights.65-0.16.hdf5")

In [9]:
predict_imgs = []
for ind in range(len(val_images)):
    predict_imgs.append(model.predict(val_images[ind:ind+1]))

ResourceExhaustedError: OOM when allocating tensor with shape[1,128,128,128,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node conv3d_5/Conv3D}} = Conv3D[T=DT_FLOAT, data_format="NDHWC", dilations=[1, 1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_conv3d_5_input_0_0/_101, conv3d_5/Conv3D/ReadVariableOp)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
predict_imgs[0].shape

In [ ]:
len(predict_imgs)

In [ ]:
subs = dataset.get_subjects(ModeKeys.TEST)[:10]

In [ ]:
c = Cleaner()
for i, s in enumerate(subs):
    image = sitk.GetImageFromArray(predict_imgs[i][0,:,:,:,0])
    t1_im = s.get_t1_image()
    image = c.resize(image, is_label_map=True, size=t1_im.GetSize())
    image.CopyInformation(s.get_t1_image())
    sitk.WriteImage(image, "/localscratch/Users/abpwrs/out/{0}.seg.nii.gz".format(i))

In [5]:
import pickle

In [7]:
with open('STRIGHT_CONV.pickle', 'wb') as f:
    a = pickle.dumps(model)

NotImplementedError: numpy() is only available when eager execution is enabled.